### 项目介绍
对垃圾邮件进行预测分类     

### 数据预处理
读取特征数据    
将一个文件存为一条数据
先只读取文本内容

首先取出所有文本     
思路为：      
匹配第一个字符为 字母的最后一个元素    
后面的所有元素    
然后拼成一个字符串   

In [65]:
import re
import os

In [102]:
# 根据路径打开文件 并提取每个邮件中的文本
def getMailTest(mailPath):
    mail = open(mailPath, "r", encoding="gb2312", errors='ignore')
    mailTextList = [text for text in mail]
    # 去除邮件头
    XindexList = [mailTextList.index(i) for i in mailTextList if re.match("[a-zA-Z0-9]", i)]
    textBegin = max(XindexList) + 1
    text = ''.join(mailTextList[textBegin:])
    # 去空格分隔符及一些特殊字符
    text = re.sub('\s+','',re.sub("\u3000","", re.sub("\n", "",text)))
    return text

In [90]:
# 通过index文件获取所有文件路径及标签值
def getPaths_Labels():
    targets = open("./trec06c/full/index", "r", encoding="gb2312", errors='ignore')
    targetList = [t for t in targets]
    newTargetList = [target.split() for target in targetList if len(target.split()) == 2]  # 分割了标记和路径
    pathList = [path[1].replace('..', './trec06c') for path in newTargetList]
    labeslList = [label[0] for label in newTargetList]
    return pathList, labeslList

In [88]:
# 获取所有文本
def getAllText(pathList):
    textList = [getMailTest(filePath) for filePath in pathList]
    return textList

### 所有文本列表
将所有文本 与目标值拼接成一个DataFrame 并对y值进行01编码

In [91]:
pathList, labeslList = getPaths_Labels()

In [92]:
textList = getAllText(pathList)

In [94]:
data_df = DataFrame(textList)
data_df['y'] = labeslList
data_df['y'] = data_df['y'].apply(lambda x:0 if x == "spam" else 1 )

In [95]:
data_df.head()

,0,y
0,非财务纠淼牟莆窆芾-（沙盘模拟）------如何运用财务岳硖岣吖芾砑ㄐ[课程背景]每一位管理...,0
1,讲的是孔子后人的故事。一个老领导回到家乡，跟儿子感情不和，跟贪财的孙子孔为本和睦。老领导的弟...,1
2,尊敬的贵公司(财务/经理)负责人您好！我是深圳金海实业有限公司（广州。东莞）等省市有分公司。...,0
3,贵公司负责人(经理/财务）您好：深圳市华龙公司受多家公司委托向外低点代开部分增值税电脑发票（...,0
4,其实cashfiesta的赚钱途径还有很多，比如游戏赚点，注册赚点等很多方法哦！当你注册以后...,0


### 训练模型
用tfidf方法训练
第一步 分词

In [19]:
import jieba
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
import pandas as pd
from pandas import DataFrame
import time

### 加载停用词 分词

In [106]:
with open('./stopwords.txt', encoding='utf8') as file:
    file_str = file.read()
    stopword_list = file_str.split('\n')
    stopword_set = set(stopword_list)

In [86]:
cutWords_list = []
i = 0
startTime = time.time()
for mail in textList:
    cutWords = [k for k in jieba.lcut(mail) if k not in stopword_list]
    i += 1
    if i % 10000 == 0:
        print('前%d篇邮件分词共花费%.2f秒' %(i, time.time()-startTime))
    cutWords_list.append(cutWords)

前10000篇邮件分词共花费71.12秒
前20000篇邮件分词共花费146.33秒
前30000篇邮件分词共花费221.58秒
前40000篇邮件分词共花费298.15秒
前50000篇邮件分词共花费368.10秒
前60000篇邮件分词共花费438.79秒


### 保存分词模型

In [93]:
with open('cutWords_list.txt', 'w',encoding="gb2312") as file: 
    for cutWords in cutWords_list:
        file.write(' '.join(cutWords) + '\n')

### tfidf向量  训练模型  预测模型

In [107]:
tfidf = TfidfVectorizer(cutWords_list, stop_words=stopword_set, min_df=40, max_df=0.3)

In [108]:
X = tfidf.fit_transform(data_df[0])
y = data_df['y']

In [111]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,data_df['y'],test_size=0.3, random_state=666)

### 建立逻辑回归模型

In [112]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
logistic_model.fit(X_train, y_train)
logistic_model.score(X_test, y_test)

0.9316001238006809

### 模型评估 混淆矩阵

In [100]:
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
y_encoded = labelEncoder.fit_transform(y)
predict_y = logistic_model.predict(X)
predict_y_decoded = labelEncoder.inverse_transform(predict_y)
from sklearn.metrics import confusion_matrix

pd.DataFrame(confusion_matrix(y, predict_y_decoded),
            columns=labelEncoder.classes_,
            index=labelEncoder.classes_)

D:\ProgramingTools\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,0,1
0,39283,3571
1,573,21193


In [113]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def eval_model(y_true, y_pred, labels):
    # 计算每个分类的Precision, Recall, f1, support
    p, r, f1, s = precision_recall_fscore_support(y_true, y_pred)
    # 计算总体的平均Precision, Recall, f1, support
    tot_p = np.average(p, weights=s)
    tot_r = np.average(r, weights=s)
    tot_f1 = np.average(f1, weights=s)
    tot_s = np.sum(s)
    res1 = pd.DataFrame({
        u'Label': labels,
        u'Precision': p,
        u'Recall': r,
        u'F1': f1,
        u'Support': s
    })
    res2 = pd.DataFrame({
        u'Label': ['总体'],
        u'Precision': [tot_p],
        u'Recall': [tot_r],
        u'F1': [tot_f1],
        u'Support': [tot_s]
    })
    res2.index = ['/']
    res = pd.concat([res1, res2])
    return res[['Label', 'Precision', 'Recall', 'F1', 'Support']]

eval_model(y, predict_y_decoded, labelEncoder.classes_)

,Label,Precision,Recall,F1,Support
0,0,0.985623,0.916671,0.949897,42854
1,1,0.855799,0.973675,0.910939,21766
/,总体,0.941894,0.935871,0.936775,64620
